# 강화학습 - 카트폴
## Matthias Plappert 라이브러리 사용


In [0]:
!git clone https://github.com/matthiasplappert/keras-rl.git 

fatal: destination path 'keras-rl' already exists and is not an empty directory.


In [0]:
%cd keras-rl

/Users/hwa-jongkim/Dropbox/lab/book5/keras-rl


In [0]:
!pwd

/Users/hwa-jongkim/Dropbox/lab/book5/keras-rl


In [0]:
!python setup.py install

running install
running bdist_egg
running egg_info
writing keras_rl.egg-info/PKG-INFO
writing dependency_links to keras_rl.egg-info/dependency_links.txt
writing requirements to keras_rl.egg-info/requires.txt
writing top-level names to keras_rl.egg-info/top_level.txt
reading manifest file 'keras_rl.egg-info/SOURCES.txt'
writing manifest file 'keras_rl.egg-info/SOURCES.txt'
installing library code to build/bdist.macosx-10.7-x86_64/egg
running install_lib
running build_py
creating build/bdist.macosx-10.7-x86_64/egg
creating build/bdist.macosx-10.7-x86_64/egg/tests
copying build/lib/tests/__init__.py -> build/bdist.macosx-10.7-x86_64/egg/tests
creating build/bdist.macosx-10.7-x86_64/egg/tests/rl
copying build/lib/tests/rl/util.py -> build/bdist.macosx-10.7-x86_64/egg/tests/rl
copying build/lib/tests/rl/test_core.py -> build/bdist.macosx-10.7-x86_64/egg/tests/rl
copying build/lib/tests/rl/__init__.py -> build/bdist.macosx-10.7-x86_64/egg/tests/rl
creating build/bdist.macosx-10.7-x86_64/egg/

removing 'build/bdist.macosx-10.7-x86_64/egg' (and everything under it)
Processing keras_rl-0.4.2-py3.6.egg
removing '/Users/hwa-jongkim/anaconda3/lib/python3.6/site-packages/keras_rl-0.4.2-py3.6.egg' (and everything under it)
creating /Users/hwa-jongkim/anaconda3/lib/python3.6/site-packages/keras_rl-0.4.2-py3.6.egg
Extracting keras_rl-0.4.2-py3.6.egg to /Users/hwa-jongkim/anaconda3/lib/python3.6/site-packages
keras-rl 0.4.2 is already the active version in easy-install.pth

Installed /Users/hwa-jongkim/anaconda3/lib/python3.6/site-packages/keras_rl-0.4.2-py3.6.egg
Processing dependencies for keras-rl==0.4.2
Searching for Keras==2.2.2
Best match: Keras 2.2.2
Adding Keras 2.2.2 to easy-install.pth file

Using /Users/hwa-jongkim/anaconda3/lib/python3.6/site-packages
Searching for scipy==1.1.0
Best match: scipy 1.1.0
Adding scipy 1.1.0 to easy-install.pth file

Using /Users/hwa-jongkim/anaconda3/lib/python3.6/site-packages
Searching for numpy==1.16.2
Best match: numpy 1.16.2
Adding numpy 

In [0]:
!pip install gym
!pip install gym[atarti]
!pip install gym[Box2D]

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [0]:
import numpy as np
import gym

from keras.models import Model
from keras.layers import Dense, Flatten, Input
from keras.optimizers import Adam

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.callbacks import ModelIntervalCheckpoint, FileLogger


Using TensorFlow backend.
/Users/hwa-jongkim/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [0]:
def build_model(state_size, num_actions):
    input = Input(shape=(1,state_size))
    x = Flatten()(input)
    x = Dense(16, activation='relu')(x)
    x = Dense(16, activation='relu')(x)
    x = Dense(16, activation='relu')(x)
    output = Dense(num_actions, activation='linear')(x)
    model = Model(inputs=input, outputs=output)
    print(model.summary())
    return model


def build_callbacks(env_name):
    checkpoint_weights_filename = 'dqn_' + env_name + '_weights_{step}.h5f'
    log_filename = 'dqn_{}_log.json'.format(env_name)
    callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, 
                                         interval=5000)]
    callbacks += [FileLogger(log_filename, interval=100)]
    return callbacks

In [0]:
# 카트폴 환경설정
ENV_NAME = 'CartPole-v0'
env = gym.make(ENV_NAME)
np.random.seed(9)
env.seed(9)
num_actions = env.action_space.n
state_space = env.observation_space.shape[0]
print(num_actions)

model = build_model(state_space, num_actions)

# 에이전트의 경험을 저장한 빠른 메모리구조, 최대크기 지정
memory = SequentialMemory(limit=50000, window_length=1)

# e 탐욕 Q 정책 설정(1~0.1, 10000 스텝)
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), 
           attr='eps', value_max=1., value_min=.1, value_test=.05,
           nb_steps=10000)

# 에이전트 생성 ()
dqn = DQNAgent(model=model, nb_actions=num_actions, 
               memory=memory, nb_steps_warmup=10,
               target_model_update=1e-2, policy=policy)
# 컴파일
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
callbacks = build_callbacks(ENV_NAME)

dqn.fit(env, nb_steps=50000,
        visualize=False,
        verbose=0,
        callbacks=callbacks)

# After training is done, we save the final weights.
dqn.save_weights('dqn_{}_weights.h5f'.format(ENV_NAME), overwrite=True)

2
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1, 4)              0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 4)                 0         
_________________________________________________________________
dense_5 (Dense)              (None, 16)                80        
_________________________________________________________________
dense_6 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_7 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_8 (Dense)              (None, 2)                 34        
Total params: 658
Trainable params: 658
Non-trainable params: 0
_________________________________________________________________
None


/Users/hwa-jongkim/Dropbox/lab/book5/keras-rl/rl/memory.py:39: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


In [0]:
dqn.test(env, nb_episodes=10, visualize=True)

Testing for 10 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200


In [0]:
!ls

CONTRIBUTING.md                   dqn_CartPole-v0_weights_35000.h5f
ISSUE_TEMPLATE.md                 dqn_CartPole-v0_weights_40000.h5f
LICENSE                           dqn_CartPole-v0_weights_45000.h5f
README.md                         dqn_CartPole-v0_weights_5000.h5f
assets                            dqn_CartPole-v0_weights_50000.h5f
build                             examples
dist                              keras_rl.egg-info
docs                              mkdocs.yml
dqn_CartPole-v0_log.json          pytest.ini
dqn_CartPole-v0_weights.h5f       rl
dqn_CartPole-v0_weights_10000.h5f setup.cfg
dqn_CartPole-v0_weights_15000.h5f setup.py
dqn_CartPole-v0_weights_20000.h5f tests
dqn_CartPole-v0_weights_25000.h5f utils
dqn_CartPole-v0_weights_30000.h5f


In [0]:
# with open('dqn_CartPloe_model_architecture.json', 'w') as f:
#     f.write(dqn.to_json())

In [0]:
dqn2 = DQNAgent(model=model, nb_actions=num_actions, 
               memory=memory, nb_steps_warmup=10,
               target_model_update=1e-2, policy=policy)
# 컴파일
dqn2.compile(Adam(lr=1e-3), metrics=['mae'])

In [0]:
dqn2.load_weights('dqn_CartPole-v0_weights_40000.h5f')

In [0]:
dqn2.test(env, nb_episodes=10, visualize=True)

Testing for 10 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
